In [2]:
# ================================
# 1. Imports & basic configuration
# ================================

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pathlib import Path

from scipy.ndimage import median_filter
from scipy.signal import find_peaks

plt.style.use("default")  # keep plots clean

# Path to the Dips + Pull-ups CSV file (relative to this notebook)
DATA_PATH = Path("..") / "data" / "Dips_PU.csv"

print("Using data file:", DATA_PATH)

# ENMO smoothing parameters
MEDIAN_FILTER_SIZE = 5          # samples
MOVING_AVERAGE_WINDOW_S = 0.2   # seconds

# Dips & pull-ups repetition detection parameters
EXERCISE_PARAMS = {
    "Dips": {
        "min_interval_s": 1.3,   # dips are faster
        "percentile": 88,        # amplitude threshold (adaptive)
    },
    "Pull-ups": {
        "min_interval_s": 2.0,   # pull-ups slower
        "percentile": 85,
    },
}


Using data file: ..\data\Dips_PU.csv


In [4]:
# =======================================
# 2. Load raw accelerometer data from CSV
# =======================================

def load_acc_csv(path: str) -> pd.DataFrame:
    """
    Load accelerometer data exported from OmGui.

    Assumes CSV has no header and columns are:
        0: timestamp (string)
        1: Ax
        2: Ay
        3: Az

    Returns a DataFrame with:
        - timestamp (datetime)
        - t (seconds from start)
        - Ax, Ay, Az (float)
    """
    df = pd.read_csv(path, header=None)
    df.columns = ["timestamp", "Ax", "Ay", "Az"]

    df["timestamp"] = pd.to_datetime(df["timestamp"])
    t0 = df["timestamp"].iloc[0]
    df["t"] = (df["timestamp"] - t0).dt.total_seconds()

    return df


df_raw = load_acc_csv(DATA_PATH)

print("Number of samples:", len(df_raw))
print("Recording time span:", df_raw["timestamp"].iloc[0], "->", df_raw["timestamp"].iloc[-1])
print("Duration (s):", df_raw["t"].iloc[-1])
df_raw.head()

Number of samples: 18000
Recording time span: 2025-11-27 19:00:06.929000 -> 2025-11-27 19:03:01.720000
Duration (s): 174.791


,timestamp,Ax,Ay,Az,t
0,2025-11-27 19:00:06.929,0.031250,-1.031250,0.093750,0.000
1,2025-11-27 19:00:06.939,0.015625,-0.171875,0.546875,0.010
2,2025-11-27 19:00:06.949,0.000000,-0.078125,0.375000,0.020
3,2025-11-27 19:00:06.960,0.000000,-0.046875,0.171875,0.031
4,2025-11-27 19:00:06.969,-0.109375,-0.109375,0.046875,0.040


In [5]:
# ===================================
# 3. Estimate sampling frequency (Hz)
# ===================================

def estimate_sampling_frequency(df: pd.DataFrame) -> float:
    dts = df["timestamp"].diff().dt.total_seconds().dropna()
    fs = 1.0 / dts.mean()
    return fs


fs = estimate_sampling_frequency(df_raw)
print(f"Estimated sampling frequency: {fs:.2f} Hz")


Estimated sampling frequency: 102.97 Hz
